## 导入库

In [2]:
import pandas as pd
from pyecharts.charts import Geo
from pyecharts.globals import ChartType
from pyecharts import options as opts
import json

## 数据清洗

In [4]:
path = r'C:\Users\yangz\AppData\Local\Programs\Python\Python310\Lib\site-packages\pyecharts\datasets\map_filename.json'
## 这是pyecharts库里面的一个文件，需要根据自己的电脑调整！！！

with open(path, encoding='utf-8') as f:
    map_names = json.load(f)

In [6]:
S_raw = pd.read_csv("c:/Users/yangz/Desktop/hadoop/job_info.csv", encoding='gbk')['工作城市']

In [7]:
def isin(a, b):
    """模糊匹配，例如黔东南苗族侗族自治州简称为黔东南也可以匹配成功"""
    for i in b:
        if a in i:
            # 如果匹配成功，就用返回True和全称
            return True, i
    return False, None

In [8]:
ignore = [] # 这些地名不在数据库中，对应的数据就被舍去了
def get_city(s):
    """把工作城市格式化为地名"""
    s = s.replace('省','').split('-')[0]
    if isin(s, map_names.keys())[0]:
        return isin(s, map_names.keys())[1]
    else:
        if s not in ignore:
            ignore.append(s)
            print(s, end=', ') # 把被忽视的地面打印出来
        return '未知地区'

## 画图

In [9]:
S = S_raw.map(get_city)
d = S.value_counts().to_dict()
del d['未知地区']
del d['普洱'] ## 不知道为什么普洱虽然在数据库中，但是画不出来
l = list(zip(d.keys(), d.values()))

异地招聘, 太仓, 昆山, 丹阳, 张家港, 义乌, 常熟, 红河州, 海宁, 泰兴, 靖江, 西昌, 燕郊开发区, 克孜勒苏柯尔克..., 

In [10]:
# 这里面异地招聘就不谈了
# 太仓、昆山、张家港等等都是县级市，而我们的统计只限于省级和市级，所以这些数据就被舍去了
# 下面的代码计算了被舍去的数据量，共计五千余条
## 如果条件允许可以做进一步考察，把这些县级市的数据算入到代管市
### 例如可以把昆山、张家港等由苏州市代管的县级市计算到苏州
S_raw.map(lambda x:x in ignore).sum()

5115

In [11]:
c = (Geo(init_opts=opts.InitOpts(width='1500px', height='950px')).add_schema(
    maptype="china-cities").add(
        series_name = '',
        data_pair = l,
        type_ = ChartType.HEATMAP,
    ).set_series_opts(label_opts=opts.LabelOpts(is_show=False)).set_global_opts(
        visualmap_opts=opts.VisualMapOpts(),
        title_opts=opts.TitleOpts(title="地理位置分布"),
    ))

In [12]:
c.render_notebook()